In [32]:
# import run_search
import run_all
import json
import pickle
import importlib
import pandas as pd
import helper
import NCBIFetch
import datetime
import NCBISearch
import PMCParse
import PUBMEDParse
# importlib.reload(PUBMEDParse)
# importlib.reload(PMCParse)
# importlib.reload(NCBIFetch)
importlib.reload(helper)
# # importlib.reload(run_search)
importlib.reload(run_all)
# import newPush
# importlib.reload(newPush)
import NCBIPush
importlib.reload(NCBIPush)

<module 'NCBIPush' from '/Users/sophierand/Desktop/Github/lit_rev_tool/NCBIPush.py'>

In [33]:
con,cur  = helper.connect_to_db()

Connecting to database
host='localhost' dbname= 'lit_rev' user='postgres' password='gres'


In [34]:
input_term = 'syndromic surveillance AND emergency room'
input_db = 'pubmed'
email_id = 'srand1@health.nyc.gov'

In [35]:
run_all.main(input_term,input_db,email_id)

In [36]:
run_all.main_push(input_term)

Connecting to database
host='localhost' dbname= 'lit_rev' user='postgres' password='gres'


In [5]:
parse_df = helper.id_run('parse')
# parse_df.head()

In [15]:
values_list = []
for index, row in new_data.iterrows():
    associated_id = row['associatedId']
    pull_source = row['pullsource']
    values_list.append((associated_id,pull_source))

In [16]:
values_list

[('28692394', 'pubmed'),
 ('25065517', 'pubmed'),
 ('26415918', 'pubmed'),
 ('28692384', 'pubmed'),
 ('28692388', 'pubmed'),
 ('24058494', 'pubmed'),
 ('28188203', 'pubmed'),
 ('28692391', 'pubmed'),
 ('28543608', 'pubmed'),
 ('22366039', 'pubmed')]

In [17]:
unique_id_inst = """INSERT INTO public.datapull_uniqueid(
    associatedid, pullsource)
    VALUES (%s, %s);
    select currval('datapull_uniqueid_uniqueid_seq');"""

In [20]:
cur.executemany(unique_id_inst,values_list)
con.commit()

In [27]:
con,cur  = helper.connect_to_db()
parse_df = helper.id_run('parse')
cur_id,cur_id_tup = NCBIPush.push_id(cur,con,input_term,parse_df)
new_data = NCBIPush.push_detail(cur,con,parse_df,cur_id,cur_id_tup)
NCBIPush.push_unique_id(cur,con,new_data)
full_df = NCBIPush.merge_ids(cur,con,new_data)
NCBIPush.push_title(cur,con,full_df)
NCBIPush.push_author(cur,con,full_df)
NCBIPush.push_keyword(cur,con,full_df)
NCBIPush.push_text(cur,con,full_df)

In [12]:
cur_id,cur_id_tup = newPush.push_id(cur,con,input_term,parse_df)

In [15]:
new_data = newPush.push_detail(cur,con,parse_df,cur_id,cur_id_tup)

In [50]:
full_df = newPush.merge_ids(cur,con,new_data)

In [69]:
# full_df.head()
newPush.push_title(cur,con,full_df)

In [23]:
# cur_unique_id_tup = cur.fetchall()
# cur_unique_id = cur_unique_id_tup[0][0]
# con.commit()

In [88]:
newPush.push_author(cur,con,full_df)

In [96]:
newPush.push_keyword(cur,con,full_df)

In [101]:
newPush.push_text(cur,con,full_df)